In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import os

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


In [2]:
import cv2
from glob import glob

IMG_SIZE = 64  # Resize images to 64x64
BATCH_SIZE = 128

def load_images(directory, img_size=IMG_SIZE):
    image_paths = glob(directory + "/*.jpg")
    images = []
    
    for img_path in image_paths:
        img = cv2.imread(img_path)  # Read image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        img = cv2.resize(img, (img_size, img_size))  # Resize
        img = (img - 127.5) / 127.5  # Normalize to range [-1, 1]
        images.append(img)
    
    return np.array(images)

# Load dataset
dataset = load_images("faces")
dataset = np.expand_dims(dataset, axis=0)  # Add batch dimension

# Create TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(dataset).shuffle(10000).batch(BATCH_SIZE)


In [3]:
def build_generator():
    model = Sequential([
        Dense(8 * 8 * 256, input_shape=(100,), activation="relu"),
        Reshape((8, 8, 256)),  # Reshape to 8x8x256

        Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(3, kernel_size=5, strides=2, activation="tanh", padding='same')
    ])
    
    return model

generator = build_generator()
generator.summary()


C:\Users\royp8\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\royp8\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16384)          │     1,654,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 16, 16, 128)    │       819,328 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 32, 32, 64)     │       204,864 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 64, 64, 3)      │         4,803 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,683,779 (10.24 MB)

 Trainable params: 2,683,779 (10.24 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
def build_discriminator():
    model = Sequential([
        Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=(64, 64, 3)),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),

        Conv2D(128, kernel_size=5, strides=2, padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),

        Flatten(),
        Dense(1, activation="sigmoid")  # Output: Probability (real or fake)
    ])
    
    return model

discriminator = build_discriminator()
discriminator.summary()


C:\Users\royp8\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │        32,769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 242,561 (947.50 KB)

 Trainable params: 242,561 (947.50 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, beta_1=0.5)


In [6]:
@tf.function
def train_step(real_images):
    # Ensure the batch size dynamically matches the input (avoids shape mismatch issues)
    batch_size = tf.shape(real_images)[0]
    
    # Generate noise for the generator
    noise = tf.random.normal([batch_size, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        # Ensure input shapes are valid before passing to the discriminator
        real_images = tf.ensure_shape(real_images, (None, 64, 64, 3))

        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Compute loss
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Compute Gradients
    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Apply Gradients
    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss


In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Hyperparameters
EPOCHS = 10000  # Adjust as needed
NOISE_DIM = 100  # Noise vector size
BATCH_SIZE = 32  # Ensure this is consistent with your dataset pipeline

# Ensure the dataset is yielding valid batches
def check_dataset(dataset):
    for batch in dataset.take(5):  # Print first 5 batches for debugging
        print("Batch Shape:", batch.shape)
        if batch.shape[1] == 0:  # If batch is empty
            print("Warning: Empty batch detected!")
            return False
    return True

# Define train_step function
@tf.function
def train_step(real_images):
    if real_images.shape[1] == 0:  # Skip empty batches
        print("Skipping empty batch")
        return 0, 0

    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Training function
def train(dataset, epochs):
    if not check_dataset(dataset):
        print("Error: Dataset contains empty batches. Please check data loading.")
        return

    for epoch in range(epochs):
        for real_images in dataset:
            print("Batch Shape in Training Loop:", real_images.shape)  # Debugging Print
            
            if real_images.shape[1] == 0:  # Skip empty batches
                print("Skipping empty batch")
                continue

            gen_loss, disc_loss = train_step(real_images)

        # Save & display progress every 500 epochs
        if epoch % 500 == 0:
            noise = tf.random.normal([16, NOISE_DIM])
            generated_images = generator(noise, training=False)

            # Show generated images
            fig, axes = plt.subplots(1, 4, figsize=(10, 10))
            for i in range(4):
                axes[i].imshow((generated_images[i] + 1) / 2)
                axes[i].axis("off")
            plt.show()

        print(f"Epoch {epoch}/{epochs}, Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}")

# Run dataset check before training
if check_dataset(train_dataset):
    train(train_dataset, EPOCHS)
else:
    print("Fix dataset before training.") 


Batch Shape: (1, 0)
Fix dataset before training.


In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Try reloading the CelebA dataset
dataset_name = "celeb_a"

data, info = tfds.load(dataset_name, split="train", as_supervised=False, with_info=True)

print("Total Samples in Dataset:", info.splits["train"].num_examples)  # Check total size

# Check if images are loading
for sample in data.take(5):
    image = sample["image"]
    print("Image shape:", image.shape)


C:\Users\royp8\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 100%|██████████| 5/5 [01:20<00:00, 16.12s/ url]


Dataset celeb_a downloaded and prepared to C:\Users\royp8\tensorflow_datasets\celeb_a\2.1.0. Subsequent calls will reuse this data.
Total Samples in Dataset: 162770
Image shape: (218, 178, 3)
Image shape: (218, 178, 3)
Image shape: (218, 178, 3)
Image shape: (218, 178, 3)
Image shape: (218, 178, 3)


In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load dataset
dataset_name = "celeb_a"
data, info = tfds.load(dataset_name, split="train", as_supervised=False, with_info=True)

# Define image size for resizing
IMG_SIZE = 64  # Change this to 128 or higher if needed
BATCH_SIZE = 128  # Adjust based on your GPU memory
BUFFER_SIZE = 10000  # For shuffling

# Preprocessing function
def preprocess(image):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize
    image = (tf.cast(image, tf.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
    return image

# Prepare dataset
dataset = data.map(lambda x: preprocess(x['image']))  # Apply preprocessing
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Check dataset output
for batch in dataset.take(1):
    print("Batch shape:", batch.shape)  # Should be (BATCH_SIZE, IMG_SIZE, IMG_SIZE, 3)


Batch shape: (128, 64, 64, 3)
